# Data Parsing, Cleansing and Integration
## Step 3
#### Author: Linda Nguyen


Environment: Python 3 and Jupyter notebook


## Introduction

In this step, we'll be given the second job advertisement dataset s3651761_dataset2.csv. We'll integrate this dataset with the dataset s3651761_dataset1_solution.csv. To do this, we'll inspect and compare the schema of s3651761_dataset1_solution.csv and s3651761_dataset2.csv to identify and resolve any schema conflicts. Finally, we'll merge and  output the integrated dataset as s3651761_dataset_integrated.csv. 


##  Import libraries 

In [1]:
import numpy as np
import pandas as pd
import datetime
import re
from IPython.display import display
import warnings
warnings.simplefilter(action ='ignore')

### 1. Examining and loading data
Examine "s3651761_dataset2.csv" the structure and schema. 

Let's have a look at the two data sets, in the following, we will give the first dataset a short name 'S1', and call the 2nd dataset 'S2'.

In [2]:
# read two datasets
df1 = pd.read_csv("s3651761_dataset2.csv")
df2 = pd.read_csv("s3651761_dataset1_solution.csv", float_precision =None)

# Display two dataframes together to compare
print('S1:' + str(df1.shape))
display(df1.head())
print('S2:' + str(df2.shape))
display(df2.head())

S1:(5000, 9)


,Location,Job Title,Monthly Payment,Closing,Category,Type,Opening,Organisation,Full-Time Equivalent (FTE)
0,Berkshire,Lead CRA UK,4583.33,2012-03-08 12:00:00,Health,NaN,2012-01-08 12:00:00,SEC Recruitment,1.0
1,Bristol,Possession Manager,2812.50,2013-09-06 12:00:00,Engineering,Permanent,2013-08-07 12:00:00,Navartis Limited,1.0
2,Coventry,NVQ Assessor Banking/Financial Services Salary...,1791.67,2013-05-02 00:00:00,Hospitality,Permanent,2013-02-01 00:00:00,Pertemps,1.0
3,Poole,HEAD OF CARE (RGN/RMN) Poole,2916.67,2013-09-24 15:00:00,Health,NaN,2013-07-26 15:00:00,Team 24 Ltd,1.0
4,Woking,SQL Server DBA,3125.00,2012-12-21 12:00:00,Information Technology,NaN,2012-11-21 12:00:00,Matchtech,1.0


S2:(134, 11)


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,70091064,Interim Compliance Consultant,South East England,InterQuest Financial Markets,non-specified,non-specified,Accounting & Finance Jobs,84000.0,2013-08-13 00:00:00,2013-10-12 00:00:00,contractjobs.com
1,70692942,ITU Nurses Required Nottinghamshire,East Midlands,Your World Healthcare,non-specified,non-specified,Healthcare & Nursing Jobs,57600.0,2013-05-12 12:00:00,2013-06-11 12:00:00,totaljobs.com
2,68801721,ICT Teacher needed for the Blackpool area ASAP,Blackpool,ITN Mark Education,non-specified,non-specified,Teaching Jobs,22000.0,2013-10-20 00:00:00,2013-12-19 00:00:00,jobserve.com
3,71224107,Adult ITU Staff Nurse Required Durham,North West England,Your World Healthcare,non-specified,non-specified,Healthcare & Nursing Jobs,57600.0,2013-12-03 00:00:00,2013-12-17 00:00:00,cv-library.co.uk
4,66965635,Locum MSK Outpatients Physiotherapist Manchester,North West England,The Placement Group,non-specified,non-specified,Healthcare & Nursing Jobs,48000.0,2012-08-03 12:00:00,2012-08-17 12:00:00,jobs.guardian.co.uk


In [3]:
print (df1.info())
print (df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Location                    5000 non-null   object 
 1   Job Title                   5000 non-null   object 
 2   Monthly Payment             5000 non-null   float64
 3   Closing                     5000 non-null   object 
 4   Category                    5000 non-null   object 
 5   Type                        3593 non-null   object 
 6   Opening                     5000 non-null   object 
 7   Organisation                4531 non-null   object 
 8   Full-Time Equivalent (FTE)  5000 non-null   float64
dtypes: float64(2), object(7)
memory usage: 351.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id          

In [4]:
# check duplicate records
duplicates = df1[df1.duplicated(keep=False)]
print ("There are "+ str(len(duplicates)) + " duplicate records found")

There are 0 duplicate records found


In [5]:
# display number of missing values
df1.isnull().sum()

Location                         0
Job Title                        0
Monthly Payment                  0
Closing                          0
Category                         0
Type                          1407
Opening                          0
Organisation                   469
Full-Time Equivalent (FTE)       0
dtype: int64

### 2. Resolving schema conflicts

Since in task 1 and task 2, we already adopt the schema of S2 as Table 1 and formated it as
in Table 2, thus schema of S2 will be similar with the global schema. Therefore, the next step is to map S1 to global schema (S2). 

Global schema: 
- `Id` - [Integer] - 8 digit integer. Note you should not change the values of the Id.

- `Title` -  [String] - If there is no title information, the value should be ‘non-specified’.

- `Location` - [String] - If there is no location information, the value should be ‘non-specified’.

- `Company`  - [String] - If there is no company information, the value should be ‘non-specified’.

- `ContractType` -[String] - It could be ‘full_time’, ‘part_time’ or ‘non-specified’.

- `ContractTime`- [String] - It could be ‘permanent’, ‘contract’ or ‘non-specified’.

- `Category`- [String]  - There are 8 possible categories: ‘IT Jobs’, ‘Healthcare & Nursing Jobs’,
‘Engineering Jobs’, ‘Accounting & Finance Jobs’, ‘Sales Jobs’, ‘Hospitality & Catering
Jobs’, ‘Teaching Jobs’, ‘PR, Advertising & Marketing Jobs’.

- `Salary`- [Float] - All the values need to be expressed to two decimal places. Also, all salary values must be valid float numbers and not null.
- `OpenDate` -[Datetime] - All the values need to be in the datetime format, yyyy-mm-dd hh:mm:ss

- `CloseDate` -[Datetime] - All the values need to be in the datetime format, yyyy-mm-dd hh:mm:ss

- `SourceName`- [String] - If there is no source information, the value should be ‘non-specified’.

#### Conflict 1: Job Title vs Title 
In S1, we have Job Title which is the Title in global schema. We'll map S1 to global schema by renaming the column.

In [6]:
# rename column 
df1.rename(columns = {'Job Title':'Title'}, inplace = True)
df1['Title'].head()

0                                          Lead CRA UK
1                                   Possession Manager
2    NVQ Assessor Banking/Financial Services Salary...
3                        HEAD OF CARE (RGN/RMN)  Poole
4                                       SQL Server DBA
Name: Title, dtype: object

#### Conflict 2: S1 has no column 'Id'
We'll use index column as Id and set generate unique 8-digits integersfor Id bu uing random library . Next convert Id from string to integer.

In [7]:
# set index colum as Id
df1.reset_index(inplace=True)
# rename column as global schema
df1 = df1.rename(columns = {'index':'Id'})

In [8]:
import random

original_ids = df1['Id'].unique()  # store unique number in original_ids
while True:
    new_ids = {id_: random.randint(10000000, 99999999) for id_ in original_ids}  # dictionary contain id
    if len(set(new_ids.values())) == len(original_ids): 
        # all the generated id's were unique
        break
    # otherwise this will repeat until they are

df1['Id'] = df1['Id'].map(new_ids)
df1['Id'].head()

0    82530642
1    75427475
2    63913041
3    72856490
4    45249310
Name: Id, dtype: int64

In [9]:
# check whether Id are integer
df1.Id.dtype

dtype('int64')

#### Conflict 3: 'Monthly Payment' vs 'Salary'
As the global schema dont have monthly salary, it has annual salary, we'll convert Monthly Payment of S1 to annual Salary as global schma. We then rename the column to Salary as global schema. 

In [10]:
# convert monthly salary to annual salary
df1['Monthly Payment'] = df1['Monthly Payment'] *12

# rename column as global schema
df1.rename(columns = {'Monthly Payment':'Salary'}, inplace = True)

In [11]:
# check again df1 
df1['Salary'].head()

0    54999.96
1    33750.00
2    21500.04
3    35000.04
4    37500.00
Name: Salary, dtype: float64

#### Conflict 4: values of  'Category' 

In [12]:
# check unique values 
df1.Category. value_counts()

Information Technology    1251
Health                     804
Engineering                732
Finance                    675
Sales                      517
Hospitality                436
Education                  341
Marketing                  244
Name: Category, dtype: int64

We can see the values of Category in S1 not match the value of Category in global schema. We need to replace these values as required: 
`Category` : ‘IT Jobs’, ‘Healthcare & Nursing Jobs’, ‘Engineering Jobs’, ‘Accounting & Finance Jobs’, ‘Sales Jobs’, ‘Hospitality & Catering Jobs’, ‘Teaching Jobs’, ‘PR, Advertising & Marketing Jobs’.

In [13]:
# create a dictionary for replace task
replace_dict ={'Information Technology':'IT Jobs', 'Health': 'Healthcare & Nursing Jobs', 'Engineering' : 'Engineering Jobs',
              'Finance' :'Accounting & Finance Jobs', 'Sales':'Sales Jobs', 'Hospitality':'Hospitality & Catering Jobs',
              'Education': 'Teaching Jobs', 'Marketing': 'PR, Advertising & Marketing Jobs'}

In [14]:
# replace values match values of global schema
df1['Category'].replace (replace_dict, inplace=True)
# check again
df1.Category. value_counts()

IT Jobs                             1251
Healthcare & Nursing Jobs            804
Engineering Jobs                     732
Accounting & Finance Jobs            675
Sales Jobs                           517
Hospitality & Catering Jobs          436
Teaching Jobs                        341
PR, Advertising & Marketing Jobs     244
Name: Category, dtype: int64

#### Conflict 5: 'Closing' and 'Opening' vs 'CloseDate' and 'OpenDate'
`OpenDate` -[Datetime] - All the values need to be in the datetime format, yyyy-mm-dd hh:mm:ss

`CloseDate` -[Datetime] - All the values need to be in the datetime format, yyyy-mm-dd hh:mm:ss

For these two columns, we only need to rename the column because they already in the required format.

In [15]:
# rename column
df1.rename(columns = {'Closing':'CloseDate'}, inplace = True)
df1.CloseDate.head(2)

0    2012-03-08 12:00:00
1    2013-09-06 12:00:00
Name: CloseDate, dtype: object

In [16]:
# rename column
df1.rename(columns = {'Opening':'OpenDate'}, inplace = True)
df1.OpenDate.head(2)

0    2012-01-08 12:00:00
1    2013-08-07 12:00:00
Name: OpenDate, dtype: object

#### Conflict 6: Organisation vs Company named,  Organisation has missing values
`Company`  - [String] - If there is no company information, the value should be ‘non-specified’.

Let's check the output of df1.info() above, there are 469 missing values in Organisation. To match the global schema we need to fill these null values with 'non-specified'. Then we will rename this attribute to Company. 

In [17]:
# fill null values with non-specified
df1['Organisation'].replace (np.nan,'non-specified', inplace=True) # np.nan is using numpy to detect missing values 

In [18]:
# rename columns 
df1.rename(columns = {'Organisation':'Company'}, inplace = True)

# check again
df1.Company.value_counts()

non-specified                      469
UKStaffsearch                       52
CVbrowser                           34
Randstad                            30
Matchtech Group plc.                28
                                  ... 
Infinitium Recruitment               1
MAIDENHALL PRIMARY SCHOOL            1
Red Personnel Ltd                    1
Alan E. Heckler, PC                  1
Radnorshire Healthy Friendships      1
Name: Company, Length: 2482, dtype: int64

#### Conflict 7: Type vs ContractTime conflict named and inconsistent values
The possible values of Type in S1 aare 'Permanent', 'Fixed Term Contract' while the values of ContractTime in global schema are 
`ContractTime` -  [String] : ‘permanent’, ‘contract’ or ‘non-specified’.

As Type contains 'Fixed Term Contract' that not listed in the global schema. We'll need to replace it as 'contract'. Then rename the atribute to ContractTime. Next, there are 1407 missing values in Type, we'll replace with 'non-specified' as global schema. 

In [19]:
# check unique values
df1.Type.value_counts()

Permanent              3034
Fixed Term Contract     559
Name: Type, dtype: int64

In [20]:
#check null
df1.isnull().sum()

Id                               0
Location                         0
Title                            0
Salary                           0
CloseDate                        0
Category                         0
Type                          1407
OpenDate                         0
Company                          0
Full-Time Equivalent (FTE)       0
dtype: int64

In [21]:
# fill null with 'non-specified' as global schema
df1['Type'].replace (np.nan,'non-specified', inplace=True)
df1['Type'].replace ('Fixed Term Contract','Contract', inplace=True)

In [22]:
# rename attribute as global schema
df1.rename(columns = {'Type':'ContractTime'}, inplace = True)

In [23]:
# double check again
df1.ContractTime.value_counts()

Permanent        3034
non-specified    1407
Contract          559
Name: ContractTime, dtype: int64

#### Conflict 8: Full-Time Equivalent (FTE) vs ContractType named and inconsistent values

As 'Full-Time Equivalent (FTE)' mention type of contract so it should related to attribute ContractType in global schema. We assumne that 1.0 is full_time, others are part_time. Thus, in this section we'll resolve the conflict by replacing values and rename the attribute. 

`ContractType`: ‘full_time’, ‘part_time’ or ‘non-specified’.

In [24]:
# check unique values
df1['Full-Time Equivalent (FTE)'].value_counts()

1.0    4857
0.2      39
0.8      36
0.6      36
0.4      32
Name: Full-Time Equivalent (FTE), dtype: int64

In [25]:
# replace values
replace_dict = {1.0:'full_time',0.2:'part_time',0.4:'part_time',0.6:'part_time',0.8:'part_time' }
df1['Full-Time Equivalent (FTE)'].replace (replace_dict, inplace=True)

In [26]:
# rename atttribute
df1.rename(columns = {'Full-Time Equivalent (FTE)':'ContractType'}, inplace = True)

In [27]:
# double check wheather it match global schema
df1.ContractType.value_counts()

full_time    4857
part_time     143
Name: ContractType, dtype: int64

#### Conflict 9: S1 don't have attribute SourceName
To resolve this conflict, we'll create a new column for S1 and concatenate it to df1. Since we have no glue the values of SourceName, we will fill the whole column with 'non-specified'.

In [28]:
# create datafrane contain only one colum SourceName
df_SourceName = pd.DataFrame({'SourceName':['non-specified']})
df_SourceName

,SourceName
0,non-specified


In [29]:
# concate df1 and df_Sourcename 
df1=pd.concat([df1,df_SourceName], axis=1)

In [30]:
# replce null values of SourceName with non-specified as required
df1.SourceName.replace(np.nan,'non-specified', inplace=True)

In [31]:
# double check whether the conflict of S1 resolve
df1.sample(3)

,Id,Location,Title,Salary,CloseDate,Category,ContractTime,OpenDate,Company,ContractType,SourceName
4253,30182129,Coscote,"Graduate Software Developer (****) Maths, C++...",32499.96,2013-08-24 12:00:00,IT Jobs,Permanent,2013-07-25 12:00:00,Monarch Recruitment Ltd.,full_time,non-specified
1849,35611361,UK,Facilities / Reception Manager,23000.04,2012-04-05 12:00:00,Accounting & Finance Jobs,Contract,2012-03-06 12:00:00,The Hood Group,full_time,non-specified
382,30106474,Merseyside,"VBNet, ASPNet, HTML5 Developer Liverpool",30500.04,2013-03-31 15:00:00,IT Jobs,Permanent,2013-01-30 15:00:00,Monarch Recruitment,full_time,non-specified


In [32]:
df2.sample(3)

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
125,70091523,Locum Occupational Therapist Community,West Midlands,The Placement Group,non-specified,non-specified,Healthcare & Nursing Jobs,43200.0,2013-05-31 15:00:00,2013-07-30 15:00:00,non-specified
62,69686619,Locum Community Respiratory Physiotherapist Bu...,South East England,The Placement Group,non-specified,non-specified,Healthcare & Nursing Jobs,5880.0,2012-03-30 15:00:00,2012-04-13 15:00:00,cvbrowser.com
12,66601865,Podiatrist Peterborough,Isle Of Wight,The Placement Group,non-specified,non-specified,Healthcare & Nursing Jobs,39360.0,2012-05-02 15:00:00,2012-07-01 15:00:00,jobsinscotland.com


#### Adapt datatypes across schemas S1 and S2
Just do double checking on the data types and make them consistent between S1 and S2 before merging.

In [33]:
print (df1.info())
print (df2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id            5000 non-null   int64  
 1   Location      5000 non-null   object 
 2   Title         5000 non-null   object 
 3   Salary        5000 non-null   float64
 4   CloseDate     5000 non-null   object 
 5   Category      5000 non-null   object 
 6   ContractTime  5000 non-null   object 
 7   OpenDate      5000 non-null   object 
 8   Company       5000 non-null   object 
 9   ContractType  5000 non-null   object 
 10  SourceName    5000 non-null   object 
dtypes: float64(1), int64(1), object(9)
memory usage: 468.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id            134 non-null    int64  
 1   Title         134 non-null

In [34]:
# now adapt the datatypes in S1 to match S2
for y in [c for c in df1.columns if c in df2.columns]: # common columns
    if df1[y].dtype != df2[y].dtype:
        print ("Column " + y + "in df1: "+ str(df1[y].dtype) + "to" + str(df2[y].dtype))
        df1[y] = df1[y].astype(df2[y].dtype) 

### 3. Merging data
We can now merge the two datasets into one.

In [35]:
# concatenate 2 dataframes
df = pd.concat([df1,df2])
print (df.shape)

# print 10 random records
df.sample(10)

(5134, 11)


,Id,Location,Title,Salary,CloseDate,Category,ContractTime,OpenDate,Company,ContractType,SourceName
811,86360771,Edinburgh,Home Manager (RGN or RMN),37500.00,2013-11-09 12:00:00,Healthcare & Nursing Jobs,non-specified,2013-10-10 12:00:00,non-specified,full_time,non-specified
4117,92098244,Crawley,Permanent Test Analyst : Crawley ****K****K,32499.96,2012-11-02 15:00:00,IT Jobs,Permanent,2012-08-04 15:00:00,Clearwater People Solutions,full_time,non-specified
2225,80609791,Eastleigh,Bank Chef Barchester Healthcare,15936.00,2013-10-05 12:00:00,Hospitality & Catering Jobs,non-specified,2013-09-05 12:00:00,Barchester Healthcare Plc,full_time,non-specified
2946,59353946,UK,Service Team Leader (After Sales),30000.00,2012-08-02 12:00:00,Engineering Jobs,Permanent,2012-06-03 12:00:00,Rise Technical Recruitment,full_time,non-specified
1955,31140277,London,MECHANICAL FITTER LONDON AND M3 CORRIDOR MOBIL...,33999.96,2013-10-11 15:00:00,Engineering Jobs,Permanent,2013-08-12 15:00:00,"Randstad Construction, Property Engineering",full_time,non-specified
4779,82192363,Oxford,Channel Account Manager Key Partner relations...,39999.96,2012-03-19 12:00:00,Sales Jobs,Permanent,2012-01-19 12:00:00,Sophos Plc,full_time,non-specified
2493,19294909,London,Freelance Principal CRA,54000.00,2013-03-16 00:00:00,Healthcare & Nursing Jobs,non-specified,2013-03-02 00:00:00,SEC Recruitment,full_time,non-specified
88,70565336,South East England,Electro Mechanical Fitter,34560.00,2013-06-03 15:00:00,Engineering Jobs,non-specified,2013-05-20 15:00:00,Experis Engineering,non-specified,TotallyExec
1459,38957299,Burnley,Web Conversion Analyst,21000.00,2012-11-30 15:00:00,IT Jobs,Permanent,2012-10-01 15:00:00,IT Works Recruitment Ltd,full_time,non-specified
4585,51168362,Wiltshire,Composite Laminator F**** Wiltshire 3 Months,28800.00,2013-06-06 12:00:00,Engineering Jobs,Contract,2013-05-23 12:00:00,Vivid Resourcing Ltd,full_time,non-specified


### 4. Resolving data conflicts:

Let's first try to check duplicates using all the records we have:

In [36]:
# check duplication
duplicates = df[df.duplicated(keep=False)] # showing all duplicated records
print ("There are "+ str(len(duplicates)) + " duplicate records found")
duplicates.sort_values(by=duplicates.columns.tolist()).head(10)

There are 0 duplicate records found


,Id,Location,Title,Salary,CloseDate,Category,ContractTime,OpenDate,Company,ContractType,SourceName


In [37]:
df.shape

(5134, 11)

Now we have 5134 rows and 11 columns without duplicate records. Is this true? Let's triple check by using global key for df. 

#### Finding global key for the data
In this senario, we can trial choose 'Company','Category','Salary' as the unique identifiers of a job advertisement record. We choose these three attributes because they are highly to be unique to identify a record. Let's check the results if any record possibly duplicated.

In [38]:
# check duplication
cols = ['Company','Category','Salary'] # this should uniquely identified a record
duplicates = df[df.duplicated(cols,keep=False)]
print ("There are "+ str(len(duplicates)) + " duplicate records found")
duplicates.sort_values(by=cols).head(10)

There are 528 duplicate records found


,Id,Location,Title,Salary,CloseDate,Category,ContractTime,OpenDate,Company,ContractType,SourceName
15,66601642,East Midlands,Community Occupational Therapist Leicestershire,45120.00,2013-06-09 12:00:00,Healthcare & Nursing Jobs,non-specified,2013-05-26 12:00:00,About Health Professionals,non-specified,MyUkJobs
51,66601684,South West England,Community Physiotherapist Bournemouth,45120.00,2012-11-10 12:00:00,Healthcare & Nursing Jobs,non-specified,2012-10-27 12:00:00,About Health Professionals,non-specified,ifaonlinejobs.co.uk
105,69869466,South East England,Band 6 Acute Occupational Therapist Oxfordshire,45120.00,2013-05-23 00:00:00,Healthcare & Nursing Jobs,non-specified,2013-04-23 00:00:00,About Health Professionals,non-specified,thegraduate.co.uk
56,66601688,East Midlands,Acute Physiotherapist Newark,47040.00,2013-08-18 00:00:00,Healthcare & Nursing Jobs,non-specified,2013-08-04 00:00:00,About Health Professionals,non-specified,cityjobs.com
65,71467354,North West England,Band 6 Intermediate Care Occupational Therapis...,47040.00,2014-02-22 00:00:00,Healthcare & Nursing Jobs,non-specified,2013-12-24 00:00:00,About Health Professionals,non-specified,michaelpage.co.uk
124,71467357,South East England,Social Service Physiotherapist Berkshire,47040.00,2013-09-02 00:00:00,Healthcare & Nursing Jobs,non-specified,2013-08-19 00:00:00,About Health Professionals,non-specified,non-specified
3631,20898549,South East England,Social Services Occupational Therapist Oxford...,48960.00,2013-07-08 00:00:00,Healthcare & Nursing Jobs,non-specified,2013-06-08 00:00:00,About Health Professionals,full_time,non-specified
4757,82818238,Cambridgeshire,Community Occupational Therapist – Cambridgeshire,48960.00,2012-02-07 12:00:00,Healthcare & Nursing Jobs,Contract,2012-01-08 12:00:00,About Health Professionals,full_time,non-specified
1651,80742389,London,Technology Trading Specialist – UNIX SQL Jav...,54999.96,2013-02-02 12:00:00,Accounting & Finance Jobs,Permanent,2012-12-04 12:00:00,Anderson Cole Recruitment Ltd,full_time,non-specified
2747,16742034,South East London,Application Support Analyst Global Brokerage ...,54999.96,2012-07-13 12:00:00,Accounting & Finance Jobs,Permanent,2012-04-14 12:00:00,Anderson Cole Recruitment Ltd,full_time,non-specified


We can see that, there are 528 duplicate records here. Let's drop them. 

In [39]:
# drop duplicate
df = df.drop_duplicates(cols, keep='last')
df.shape

(4811, 11)

### 5. Saving the integrated and reshaped data
This last part of the integration process is to export our output data to csv format, named as:
- 's3651761_dataset_integrated.csv'

In [40]:
df.to_csv("s3651761_dataset_integrated.csv",index = False, float_format='%.2f')

## Summary of project
In project, we have demonstrated process of data integration.
- All schema conflicts are identified, explained and resolved, we then simply identify duplicate using the complete records. 
- Carefully, we examine and identify global key for identifying data conflicts in integration.